In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import gmaps
from config import gkey
import requests
import json


In [2]:
#defining ideal conditions
temp_max = 35
temp_min = 20
humidity_max=40
cloud_max=50
wind_max=10 #in meters/sec

In [3]:
filename = "Data_and_Output/weather.csv"
destination_df = pd.read_csv(filename, dtype = str)
destination_df = destination_df.astype({"Lat":"float64","Long":"float64", "Temp":"float64", "Humidity":"float64",
                                        "%Cloud":"float64","Wind_Speed":"float64"})
print(list(destination_df))
print(destination_df.shape)      

['City', 'Country', 'Lat', 'Long', 'Temp', 'Humidity', '%Cloud', 'Wind_Speed', 'Hemisphere']
(501, 9)


In [4]:
#creating heat map of humidity
gmaps.configure(gkey)

locations = destination_df[["Lat", "Long"]]
humidity = destination_df["Humidity"]


hum_max = humidity.max()
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '4px'
}

fig = gmaps.figure(layout = figure_layout, map_type="TERRAIN", center = [0,0], zoom_level =1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating = False, max_intensity = hum_max,  
                                 point_radius=4)
heat_layer.gradient = ["aqua","green", "yellow","red"]  #color codes from (https://www.w3.org/TR/css-color-3/#html4)
heat_layer.opacity = 0.3
fig.add_layer(heat_layer)

print("Heat Map Showing Humidity Variation at 500 Locations Chosen Randomly")

fig

Heat Map Showing Humidity Variation at 500 Locations Chosen Randomly


Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='4px', width='600px'))

In [5]:
#filtering data


destination_df = destination_df.loc[(destination_df["Temp"]<=temp_max)&(destination_df["Temp"]>=temp_min)]
destination_df = destination_df.loc[(destination_df["Humidity"]<=humidity_max)]
destination_df = destination_df.loc[(destination_df["%Cloud"]<=cloud_max)]
destination_df =  destination_df.loc[(destination_df["%Cloud"]<=wind_max)]
destination_df

,City,Country,Lat,Long,Temp,Humidity,%Cloud,Wind_Speed,Hemisphere
18,sakakah,sa,29.0129,42.8145,30.00,23.0,0.0,6.20,Northern
52,carnarvon,au,-19.4920,109.7165,23.00,27.0,0.0,3.60,Southern
101,talavera de la reina,es,39.6201,-4.5537,24.77,31.0,0.0,2.60,Northern
106,urumqi,cn,42.8242,89.4065,29.00,26.0,0.0,2.00,Northern
109,sidi ali,dz,35.0556,0.2564,24.29,38.0,0.0,1.85,Northern
127,yumen,cn,43.3560,97.5002,25.35,31.0,0.0,6.75,Northern
169,omsukchan,ru,60.6001,155.7842,27.50,27.0,0.0,2.33,Northern
230,aswan,eg,22.3465,28.9367,29.99,21.0,0.0,3.64,Northern
235,khandyga,ru,64.1536,138.2116,29.64,32.0,0.0,2.15,Northern
292,broome,au,-24.1441,124.2337,25.00,16.0,0.0,6.70,Southern


In [6]:
destination_df.shape

(22, 9)

In [34]:
destination_df = destination_df.astype({"Lat":"object","Long":"object"})



loc_x = [destination_df.iloc[i][2] for i in range(0,len(destination_df))]
loc_y = [destination_df.iloc[i][2] for i in range(0,len(destination_df))]
loc = [(f"{loc_x[i]} , {loc_y[i]}") for i in range(0,len(loc_x))]
#locations = ((destination_df.iloc[i][2]&","&destination_df.iloc[i][3]) for i in range(0,len(destination_df)))

#target_search="Hotel"
target_radius = 10000
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


m=0 #m to track exceptions if needed
hotel = []
lat = []
long = []
for i in range(0,len(loc)):
            params = {
                   "location":loc[i],
                   "radius":target_radius,
                   "type":target_type,
                   "key":gkey
               }
            try:
                response = requests.get(base_url, params=params)
                data=response.json()
                
            except KeyError:
                m+=1
            if data["status"]!= "ZERO_RESULTS":
                hotel.append(data["results"][0]["name"])
                lat.append((data["results"][0]["geometry"]["location"]["lat"]))
                long.append((data["results"][0]["geometry"]["location"]["lng"]))
                
            else:
                hotel.append("No hotel in 10 km")
                lat.append((loc_x[i]))
                long.append((loc_y[i]))
                
         
            
            

In [35]:
print(hotel)
print(lat)

['الزعيم للخدمات والتسويق', 'No hotel in 10 km', 'Üzümlü', 'Lentekhi', 'No hotel in 10 km', 'Karasu', "Ivdel'", 'No hotel in 10 km', 'No hotel in 10 km', 'No hotel in 10 km', 'No hotel in 10 km', 'Gbc E-Lounge', 'No hotel in 10 km', 'No hotel in 10 km', 'No hotel in 10 km', 'Naushibay', 'Musabeyli', 'Doanh Nghiệp Tư Nhân Phú Hào', 'No hotel in 10 km', 'No hotel in 10 km', 'Sivrikaya', 'Kabardino-Balkarskiy Vysokogornyy Gosudarstvennyy Zapovednik']
[28.97493749999999, -19.492, 39.71029799999999, 42.7931509, 35.0556, 43.2953641, 60.6970661, 22.3465, 64.1536, -24.1441, 34.8356, 19.143315, -23.4123, -20.9829, -14.9995, 47.7759204, 36.885897, 20.86116729999999, 63.7705, 27.3638, 40.680068, 43.07396509999999]


In [41]:
loc_vacation = []
for i in range(0,len(hotel)):
    if hotel[i]!= "No hotel in 10 km":        
        loc_x = lat[i]
        loc_y = long[i]
        loc_vacation.append((loc_x, loc_y))

#loc_vacation = [(-19,22 )]
fig1 = gmaps.figure(layout = figure_layout, map_type="TERRAIN", center = [0,0], zoom_level =1)
markers = gmaps.marker_layer(loc_vacation)
fig1.add_layer(markers)
fig1

Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='4px', width='600px'))

In [28]:
loc_x[1]

'-19.492'

In [18]:
loc_y[1]

22

In [42]:
len(loc_vacation)

11